In [ ]:
%%HTML
<p id = 'heading1_p'> ECC608 TrustMANAGER with keySTREAM™ in-field Provisioning and key management </p>
<hr id = 'hline'> </hr> 
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p>Transport Layer Security 1.2 (TLS 1.2) has become the de facto standard for connecting embedded systems to a network. While TLS 1.2 is undeniably robust, an embedded system still requires a unique, secure and trusted identity to prevent large-scale remote attacks. It’s relies heavily on how well protected is your private key inside the embedded system. </p>
<p>Securing communication with a Cloud service and manipulating keys comes with many challenges: storing and using keys in the microcontroller exposes them, operating systems and software have bugs, the Heartbleed bug for OpenSSL was notable by easily exposing keys. When combined with the TLS 1.2 protocol, Microchip’s ECC608 CryptoAuthentication™ device offers a unique, trusted, and verifiable identity that can help protect billions of connected devices by physically isolating keys and secrets from the application code.​</p>
<p>This use case demonstrates the pre-provisioned ECC608 TrustMANAGER device and the keySTREAM™ SaaS in-field provisioning and key management. The ECC608 will go through a take ownership process where the keySTREAM will set a custom chain of certificate from the SaaS and key the root and intermediate private keys protected in Kudelski cloud HSMs.​</p>
</div>

<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p style="float: left; font-size: 9pt; text-align: center; width: 36.5%; margin-right: 1%;">
    <img src="assets/images/ks_slot_Information.png" alt="Cryptographic Assets" style="width: 100%"><b>Cryptographic Asset list</b></p>
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 40%; margin-right: 1%;">
    <img src="assets/images/ks_aws_usecase_steps.png" alt="Use Case Steps" style="width: 99.99%">
    <b>Use Case Implementation</b><br/></p>
<p style="clear: both;">
</div>
</div>

<p id = 'heading1_p' style='margin-top:40px; font-size: 20px; font-weight: bold;'>Use Case Information</p>
<hr id = 'hline'> </hr> 
<div id='div_element'>
<p> The Transaction Diagram represent all steps to be performed to successfully implement the given Use Case, during development and after product delivery to the end-user. </p>
<p> For the ease of the execution, when steps are executed from transaction diagram, these are emulated in the Host-PC rather than executed in the 
Host-Microcontroller/Processor from the evaluation kit. The MPLAB X project/C Source code includes software code to be executed in your Host-Microcontroller/Processor 
in your embedded project. </p>

<p>Select your Evaluation kit and then before clicking on each step to execute the Use Case please complete the Pre Usecase transaction steps mentioned in the <strong>Usecase Help</strong>. Click on <strong>Usecase Help</strong> button in the transaction diagram for additional details.</p>

</div>

In [ ]:
import os, shutil
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(curr_path, 'assets', 'python')])

# Set usecase working directory
tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'keystream_connect')
os.makedirs(usecase_working_dir, exist_ok=True)

# Add the boards that can perform this Usecase
boards = ProtoBoard(max_width=400)
boards.add_board(
    'EV10E69A',
    os.path.join(curr_path,'..','backend','boards', 'EV10E69A', 'EV10E69A_WINC.png'),
    os.path.join(curr_path,'..','backend','boards', 'EV10E69A', 'EV10E69A.hex'),
    os.path.join(curr_path, 'firmware', 'd21_aws', 'keystream_aws_d21.X')
)
boards.render_boards(0)

# Copying files to designated place
target_winc_path = os.path.join(tp_settings.get_base_folder(), 'winc_firmware_upgrade')
if not os.path.exists(target_winc_path):
    shutil.copytree(os.path.join(get_board_path(), 'DM320118', 'winc_firmware_upgrade'), target_winc_path)
if not os.path.isfile(os.path.join(tp_settings.get_base_folder(), 'aws_credentials.yaml')):
    shutil.copy(os.path.join(curr_path, 'assets', 'aws_credentials.yaml'), tp_settings.get_base_folder())
if not os.path.isfile(os.path.join(tp_settings.get_base_folder(), 'keystream_config.yaml')):
    shutil.copy(os.path.join(curr_path, 'assets', 'keystream_config.yaml'), tp_settings.get_base_folder())
if not os.path.isfile(os.path.join(tp_settings.get_base_folder(), 'aws-zero-touch-full-setup.yaml')):
    shutil.copy(os.path.join(curr_path, 'assets', 'aws-zero-touch-full-setup.yaml'), tp_settings.get_base_folder())
if not os.path.isfile(os.path.join(tp_settings.get_base_folder(), 'aws-zero-touch-full-setup-cn.yaml')):
    shutil.copy(os.path.join(curr_path, 'assets', 'aws-zero-touch-full-setup-cn.yaml'), tp_settings.get_base_folder())
target_lite_manifest_path = os.path.join(tp_settings.get_base_folder(), 'keystream_connect', 'lite_manifest')
if not os.path.exists(target_lite_manifest_path):
    shutil.copytree(os.path.join(curr_path, 'assets', 'lite_manifest'), target_lite_manifest_path)

# Import this module after copying necessary files
import keystream_connect
import winc_fw_upgrade

keySTREAM_AWS_Cert_Mgmt_Instance = keystream_connect.keySTREAM_AWS_Cert_Mgmt(boards, TPSettings().get_base_folder(), os.path.join(TPSettings().get_base_folder(), 'keystream_connect'))
winc_update_inst = winc_fw_upgrade.WincFirmwareUpdater(boards)

usecase_td = UsecaseDiagram(os.path.join(curr_path, 'assets', 'images', 'ks_aws_ud.png'), working_dir=usecase_working_dir)
usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))

generate_manifest = usecase_td.add_script(76.5,62.9,31,29, keySTREAM_AWS_Cert_Mgmt_Instance.manifest_lite)
pre_config_inputs = usecase_td.add_script(76.5,91.9,38,34, keySTREAM_AWS_Cert_Mgmt_Instance.get_inputs)
download_ca_cert = usecase_td.add_script(76.5,121.9,35,30, keySTREAM_AWS_Cert_Mgmt_Instance.get_cacert, prereq_scripts = [pre_config_inputs])
get_registration_code = usecase_td.add_script(76.5,158.9,35,30, keySTREAM_AWS_Cert_Mgmt_Instance.get_aws_code, prereq_scripts = [download_ca_cert])
download_pop_cert = usecase_td.add_script(76.5,197.9,35,30, keySTREAM_AWS_Cert_Mgmt_Instance.get_pop, prereq_scripts = [get_registration_code])
upload_cert = usecase_td.add_script(76.5,235.9,35,30, keySTREAM_AWS_Cert_Mgmt_Instance.register_cert_to_aws, prereq_scripts = [download_pop_cert])
winc_firmware_upgrade = usecase_td.add_script(760,00,284,25, winc_update_inst.upgrade_winc1500)

usecase_td.add_link(210,52,225,15,
    os.path.join(curr_path, 'assets', 'keySTREAM AWS Connect.md'))

usecase_td.add_usecase_help(os.path.join(curr_path, 'assets', 'keySTREAM AWS Connect.md'))

display(boards.canvas)
usecase_td.display_canvas()

In [ ]:
%%HTML
<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Conclusion</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p>When ready to start the Secure Exchange Process, please fill the Secure Exchange Process questionnaire and start a support ticket (attached the questionnaire output “.yml” file from your download folder).
Microchip will then provide you with a Custom Part Number.</p>
<p>Populate the User data and keys in the configurator and “Generate Provisioning Package” under the “Production Tab”. 
Once done, please attach the .zip file generated from the Production Encrypted Provisioning Package button and attach it to the support case.</p>
<p>Microchip will then enable the procurement of Verification Samples before starting mass-production.</p>
</div>
</div>